In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os
import heapq

In [2]:
def printcsv(data):
    out  = open("sub.csv", "w")
    out.write("RowId,prediction\r\n")
    for i,elt in enumerate(data):
        out.write(str(i+1)+","+elt+"\r\n")
    print("done")
    out.close()

def readFile(name):
   # filepath = "../input/"+name.lower()+".csv/"+name.upper()+".CSV"
    filepath = "/home/zvergne/Documents/5IF/FouillesDeDonnees/Stardig/input/"+name.lower()+".csv/"+name.upper()+".CSV"
    maxNbColumn = -1
    test = "test" in name
    with open(filepath) as fp:
        line = fp.readline()
        while line:
            length = len(line.split(","))
            if (length> maxNbColumn):
                maxNbColumn =length
            line = fp.readline()

    if("test" in name):
        columns = ["Race"] + list(range(0, maxNbColumn-1))
    else:
        columns = ["IDplayer", "Race"] + list(range(0, maxNbColumn-2))

    return  pd.read_csv(filepath, names=columns, engine='python')

def isAction(strAction):
    if not strAction :
        return False
    return strAction == "s" or strAction == "Base" or strAction.startswith("hotkey") or strAction.startswith("t")
def getNbOfHotKey(strAction):
    if strAction.startswith("hotkey"):
        return (int(strAction[6]),int(strAction[7]))
    else:
        return (-1,-1)
def isTime(strAction):
    if not strAction:
        return False
    return strAction[0]=="t"
def getTime(strAction):
    strAction.pop(0)
    return int(strAction)

In [ ]:

def putIdOnTest(trainData,testData, doAnEstimation=False):
    print("bEGIN")
    #features = []
   # training = trainData
    training = trainData.apply(lambda row: pd.Series(dictFeatures(row)), axis=1)

    training_answer = trainData["IDplayer"]

    clf = ExtraTreesClassifier(n_estimators=100).fit(training, training_answer)

    X_train, X_test, y_train, y_test = train_test_split(training, training_answer, random_state=0)
    print('Accuracy of Decision Tree classifier on training set: {:.4f}'.format(clf.score(X_train, y_train)))
    print('Accuracy of Decision Tree classifier on all set: {:.4f}'.format(clf.score(training, training_answer)))
    for i,elt in enumerate(clf.predict(training)):
        if(training_answer[i]!=elt):
            print("Error")
            print(elt)
            print(training_answer[i])
            print("Play:")
            print(trainData.iloc[[2]])
            print("All the play of the player")
            print(trainData.loc[trainData['IDplayer'] == elt])
            print("All the play of the answer")
            print(trainData.loc[trainData['IDplayer'] == training_answer[i]])
    print('Accuracy of Decision Tree classifier on test set: {:.4f}'.format(clf.score(X_test, y_test)))
    test = testData.apply(lambda row: pd.Series(dictFeatures(row)), axis=1)

    #print(test)
    return clf.predict(test)

In [4]:
import numpy as np

def dictFeatures(row):
    dictF = {'Zerg': row['Race']=="Zerg","Terran": row["Race"]=="Terran"}
    nbActionIn5FirstSec = 0
    singleMin = False
    for action in row:
        if action == "SingleMineral":
            singleMin = True
        if not isAction(action):
            continue
        if action.startswith("t"):
            break
        nbActionIn5FirstSec += 1
    dictF["SingleMineralIn5FirstSec"] =  singleMin
    dictF["nbActionsIn5FirstSec"] = nbActionIn5FirstSec
    favHotKey = 1
    tabHotKeyUsage = [0] * 10
    firstHK = -1
    secondHK = -1
    thirdHK = -1
    fourthHK = -1
    HKspressed = [-1] * 4
    timeHKpressed = [-1] * 4
    howMuchHKDefBeforeAction = 0
    howMuchHKDefBeforeActionRes = -1
    currTime = 0
    for action in row:
        if isAction(action):
            nbHK = getNbOfHotKey(action)

            if (nbHK[0] != -1):
                tabHotKeyUsage[nbHK[0]] += 1
                if nbHK[1] != 0 and howMuchHKDefBeforeActionRes == -1:
                    howMuchHKDefBeforeActionRes = howMuchHKDefBeforeAction
                else:
                    howMuchHKDefBeforeAction +=1
                for i in range(4):
                    if HKspressed[i]==-1 and (i>0 and nbHK[0]!=HKspressed[i-1]):
                        HKspressed[i]= nbHK[0]
                        timeHKpressed[i] = currTime
        elif isTime(action):
            currTime = getTime(action)
        elif not action:
            break


    for i,key in enumerate(HKspressed):
        dictF[str(i)+"HK"] = key
        dictF[str(i)+"HKtime"] = timeHKpressed[i]


    dictF["howMuchHKDefBeforeAction"] = howMuchHKDefBeforeActionRes




    for i in [0,9,8,7,6]:
        dictF["useHK"+str(i)+"MoreThan 10 times"] = tabHotKeyUsage[i] >10
        dictF["useHK"+str(i)] = tabHotKeyUsage[i] >0
    dictF["favHK"] = tabHotKeyUsage.index(max(tabHotKeyUsage))
    nlargest = heapq.nlargest(2, tabHotKeyUsage)
    dictF["secfavHK"] = tabHotKeyUsage.index(nlargest[1])
    tabHotKeyUsage = [0] * 10
    for action in row:
        if isAction(action):
            nbHK = getNbOfHotKey(action)
            if (nbHK[0] != -1):
                tabHotKeyUsage[nbHK[0]] += 1
        elif isTime(action):
            currTime = getTime(action)
            if currTime > 100:
                break

    for i,key in enumerate(tabHotKeyUsage):
        dictF[str(i)+"proportionInBegin"] = key
    lastHK = -1
    d = np.zeros((10,10))
    for i in range(len(row)):
        if not row[i]:
            break
        if isAction(action):
            nbHK = getNbOfHotKey(action)
            if lastHK !=-1:
                d[lastHK][nbHK[0]]+=1
            lastHK = nbHK[0]
    for i in range(10):
        sumRow = 0
        for j in range(10):
            sumRow += d[i][j]
        for j in range(10):
            if sumRow!=0:
                d[i][j] = d[i][j]/sumRow
            else:
                d[i][j] = 0
            dictF["freq"+str(i)+str(j)]=d[i][j]
        

#        actionList.append([row[i], row[i+1], row[i+2]])
    #Detect most frequent trigram
#    actionList= []
#    for i in range(len(row)-3):
#        if not row[i] or not row[i+1] or not row[i+2]:
#            break
#        actionList.append([row[i], row[i+1], row[i+2]])

#    maxActions = []
#    maxFrequency = -1
#    for actions in actionList:
#        frequency= actionList.count(actions)
#        if maxFrequency < frequency:
#            maxActions = actions
#           maxFrequency = frequency
    #actions.sort()
#    dictF["freqngram"]=maxFrequency
    return dictF


In [8]:
#putIdOnTest(readFile("train"),readFile("test"))

In [ ]:
printcsv(putIdOnTest(readFile("train"),readFile("test")))

In [5]:
def dispPlayplayer(data):
    #print(data.IDplayer.unique())
    for player in data.IDplayer.unique():
      #  print("Player"+player)
        #if player == "http://eu.battle.net/sc2/en/profile/4234852/1/First/":
        print(data.loc[data['IDplayer']== player].describe())
        #for elt in data.loc[data['IDplayer'] == player]:
            #if "ChatEvent" in str(elt):
         #   print(elt)


In [ ]:
dispPlayplayer(readFile("train_long"))

                                                 IDplayer     Race  \
count                                                  15       15   
unique                                                  1        1   
top     http://eu.battle.net/sc2/en/profile/4234852/1/...  Protoss   
freq                                                   15       15   

                                    0                        1  \
count                              15                       15   
unique                             13                        6   
top     5:CameraEvent:20.0:18.7578125  16:SelectionEvent:Nexus   
freq                                2                        6   

                                      2                              3  \
count                                15                             15   
unique                                6                              8   
top     17:BasicCommandEvent:TrainProbe  21:SelectionEvent:Probe;Probe   
freq                  

                                                 IDplayer  Race  \
count                                                  26    26   
unique                                                  1     1   
top     http://eu.battle.net/sc2/en/profile/3538115/1/...  Zerg   
freq                                                   26    26   

                                      0                       1  \
count                                26                      26   
unique                               19                      11   
top     5:CameraEvent:28.0:135.75390625  16:SelectionEvent:Lair   
freq                                  3                       8   

                                          2                                3  \
count                                    26                               26   
unique                                    9                                7   
top     17:SelectionEvent:Larva;Larva;Larva  17:BasicCommandEvent:MorphDrone   
freq    

                                                 IDplayer  Race  \
count                                                  16    16   
unique                                                  1     1   
top     http://eu.battle.net/sc2/en/profile/2222468/1/...  Zerg   
freq                                                   16    16   

                                      0                       1  \
count                                16                      16   
unique                               16                       9   
top     6:CameraEvent:116.0:137.7578125  8:SelectionEvent:Drone   
freq                                  1                       6   

                                     2                                3  \
count                               16                               16   
unique                              15                               15   
top     9:BasicCommandEvent:MorphDrone  10:BasicCommandEvent:MorphDrone   
freq                        

                                                 IDplayer  Race  \
count                                                  15    15   
unique                                                  1     1   
top     http://eu.battle.net/sc2/en/profile/251061/1/L...  Zerg   
freq                                                   15    15   

                                     0                       1  \
count                               15                      15   
unique                              13                      13   
top     5:CameraEvent:28.0:135.7578125  16:SelectionEvent:Lair   
freq                                 3                       2   

                                          2                                3  \
count                                    15                               15   
unique                                   10                               11   
top     20:SelectionEvent:Larva;Larva;Larva  21:BasicCommandEvent:MorphDrone   
freq         

                                                 IDplayer     Race  \
count                                                  25       25   
unique                                                  1        1   
top     http://eu.battle.net/sc2/en/profile/1139573/1/...  Protoss   
freq                                                   25       25   

                                    0                        1  \
count                              25                       25   
unique                             21                       10   
top     5:CameraEvent:26.0:24.7578125  14:SelectionEvent:Nexus   
freq                                2                        7   

                                      2                         3  \
count                                25                        25   
unique                               10                        17   
top     18:BasicCommandEvent:TrainProbe  22:ControlGroupEvent:5:0   
freq                                  6   

...

                                                 IDplayer     Race  \
count                                                  19       19   
unique                                                  1        1   
top     http://eu.battle.net/sc2/en/profile/377576/1/L...  Protoss   
freq                                                   19       19   

                                    0                        1  \
count                              19                       19   
unique                             16                        6   
top     5:CameraEvent:20.0:18.7578125  13:SelectionEvent:Nexus   
freq                                2                        7   

                                      2                              3  \
count                                19                             19   
unique                                5                              8   
top     13:BasicCommandEvent:TrainProbe  17:SelectionEvent:Probe;Probe   
freq                  

                                                 IDplayer  Race  \
count                                                   7     7   
unique                                                  1     1   
top     http://eu.battle.net/sc2/en/profile/219517/1/T...  Zerg   
freq                                                    7     7   

                                      0                                    1  \
count                                 7                                    7   
unique                                7                                    7   
top     6:CameraEvent:116.0:137.7578125  20:SelectionEvent:Larva;Larva;Larva   
freq                                  1                                    1   

                                      2  \
count                                 7   
unique                                7   
top     21:BasicCommandEvent:MorphDrone   
freq                                  1   

                                               

                                                 IDplayer  Race  \
count                                                  14    14   
unique                                                  1     1   
top     http://eu.battle.net/sc2/en/profile/2639840/1/...  Zerg   
freq                                                   14    14   

                                    0                                 1  \
count                              14                                14   
unique                             13                                13   
top     6:CameraEvent:97.0:14.7578125  14:CameraEvent:100.5:153.7578125   
freq                                2                                 2   

                             2                                    3  \
count                       14                                   14   
unique                      12                                    7   
top     17:SelectionEvent:Hive  18:SelectionEvent:Larva;Larva;Larva   
freq

                                                 IDplayer     Race  \
count                                                  12       12   
unique                                                  1        1   
top     http://eu.battle.net/sc2/en/profile/349650/1/H...  Protoss   
freq                                                   12       12   

                                     0                       1  \
count                               12                      12   
unique                              12                       7   
top     8:CameraEvent:171.0:17.7578125  9:SelectionEvent:Probe   
freq                                 1                       3   

                                      2                                3  \
count                                12                               12   
unique                               10                                8   
top     12:BasicCommandEvent:TrainProbe  25:BasicCommandEvent:TrainProbe   
freq          

                                                 IDplayer     Race  \
count                                                  10       10   
unique                                                  1        1   
top     http://eu.battle.net/sc2/en/profile/3971497/1/...  Protoss   
freq                                                   10       10   

                                     0                        1  \
count                               10                       10   
unique                              10                        7   
top     9:CameraEvent:161.0:21.7578125  24:SelectionEvent:Nexus   
freq                                 1                        2   

                                      2                              3  \
count                                10                             10   
unique                                6                              9   
top     22:BasicCommandEvent:TrainProbe  26:SelectionEvent:Probe;Probe   
freq             